# 🏥 Heart Disease Prediction - V17 Ensemble

## **9-Model Ensemble with 3 Seeds + Blending Strategy**

### Key Features:
- **3 seeds per algorithm:** [42, 123, 2024] for maximum diversity
- **9 models total:** LGB+CAT+XGB × 3 seeds
- **5-fold CV** with domain feature engineering
- **Target encoding:** 7 categorical features (smoothing=30)
- **Blend with V16:** Combines new 9-model ensemble with previous best (A.csv)
- **Multiple blend ratios:** Tests 0.3, 0.4, 0.5, 0.6 weighting

### Results:
- **Best new ensemble AUC:** 0.955442 (meta-model)
- **Kaggle Score:** 0.95360
- **Status:** ⭐ Advanced blending strategy

## Step 1: Load Data & Previous Best Submission

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Load datasets
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

# Load previous best submission (V16)
prev_sub = pd.read_csv('submission.csv')  # V16 submission
prev_preds = prev_sub.set_index('id')['Heart Disease'].values
print(f"Previous submission loaded: {prev_sub.shape}")
print(f"Previous submission range: [{prev_preds.min():.4f}, {prev_preds.max():.4f}]")

target_column = 'Heart Disease'
X_train = train_df.drop([target_column, 'id'], axis=1)
y_train = train_df[target_column].map({'Absence': 0, 'Presence': 1})
X_test  = test_df.drop('id', axis=1)
test_ids = test_df['id'].values

X_train = X_train.fillna(X_train.median()).reset_index(drop=True)
X_test  = X_test.fillna(X_test.median()).reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print(f"Train: {X_train.shape} | Test: {X_test.shape}")

## Step 2: Domain Feature Engineering + Target Encoding

In [ ]:
def engineer_features(df):
    df = df.copy()
    df['cp_asymptomatic']  = (df['Chest pain type'] == 4).astype(int)
    df['st_significant']   = (df['ST depression'] > 2.0).astype(int)
    df['max_hr_ratio']     = df['Max HR'] / (220 - df['Age'])
    df['hr_reserve']       = (220 - df['Age']) - df['Max HR']
    df['has_vessels']      = (df['Number of vessels fluro'] > 0).astype(int)
    df['thal_reversible']  = (df['Thallium'] == 7).astype(int)
    df['thal_fixed']       = (df['Thallium'] == 6).astype(int)
    df['angina_st']        = df['Exercise angina'] * df['ST depression']
    df['cp_angina']        = df['Chest pain type'] * df['Exercise angina']
    df['age_male']         = df['Age'] * df['Sex']
    df['vessels_thal']     = df['Number of vessels fluro'] * df['Thallium']
    df['st_slope_combo']   = df['ST depression'] * df['Slope of ST']
    df['risk_score']       = (
        df['cp_asymptomatic'] +
        df['st_significant'] +
        df['has_vessels'] +
        df['thal_reversible'] +
        df['Exercise angina'] +
        (df['ST depression'] > 1.0).astype(int) +
        (df['max_hr_ratio'] < 0.7).astype(int)
    )
    df['age_group'] = pd.cut(df['Age'], bins=[0,45,55,65,100], labels=[0,1,2,3]).astype(int)
    df['high_bp']   = (df['BP'] > 140).astype(int)
    df['high_chol'] = (df['Cholesterol'] > 240).astype(int)
    df['fbs_age']   = df['FBS over 120'] * df['Age']
    return df

X_train = engineer_features(X_train)
X_test  = engineer_features(X_test)

TE_COLS = ['Chest pain type', 'Thallium', 'Slope of ST',
           'Number of vessels fluro', 'EKG results',
           'risk_score', 'age_group']

def target_encode(X_tr, y_tr, X_val, X_te, cols, smoothing=30):
    gm = y_tr.mean()
    X_tr_e, X_val_e, X_te_e = X_tr.copy(), X_val.copy(), X_te.copy()
    for col in cols:
        stats = y_tr.groupby(X_tr[col]).agg(['sum','count'])
        sm = (stats['sum'] + gm * smoothing) / (stats['count'] + smoothing)
        X_tr_e[f'{col}_te']  = X_tr[col].map(sm).fillna(gm)
        X_val_e[f'{col}_te'] = X_val[col].map(sm).fillna(gm)
        X_te_e[f'{col}_te']  = X_te[col].map(sm).fillna(gm)
    return X_tr_e, X_val_e, X_te_e

print(f"Features engineered and ready for encoding")

## Step 3: Train 9 Models with 3 Seeds (LGB+CAT+XGB)

In [ ]:
n_folds = 5
SEEDS   = [42, 123, 2024]  # 3 seeds for more diversity
skf     = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

all_oof   = []
all_test  = []
all_scores = []

print(f"Training 9 models × {n_folds} folds")
print(f"Est. runtime: ~8 hours\n")

for seed in SEEDS:

    # --- LightGBM ---
    oof_lgb  = np.zeros(len(X_train))
    test_lgb = np.zeros(len(X_test))
    sc_lgb   = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_tr_e, X_val_e, X_te_e = target_encode(
            X_train.iloc[tr_idx], y_train.iloc[tr_idx],
            X_train.iloc[val_idx], X_test, TE_COLS)

        m = lgb.LGBMClassifier(
            n_estimators=10000, learning_rate=0.01, subsample=0.7,
            colsample_bytree=0.7, min_child_samples=50, reg_alpha=0.1,
            reg_lambda=5.0, num_leaves=31, random_state=seed, n_jobs=-1, verbose=-1
        )
        m.fit(X_tr_e, y_train.iloc[tr_idx],
              eval_set=[(X_val_e, y_train.iloc[val_idx])],
              callbacks=[lgb.early_stopping(200, verbose=False),
                         lgb.log_evaluation(-1)])
        oof_lgb[val_idx] = m.predict_proba(X_val_e)[:, 1]
        test_lgb += m.predict_proba(X_te_e)[:, 1] / n_folds
        sc_lgb.append(roc_auc_score(y_train.iloc[val_idx], oof_lgb[val_idx]))

    all_oof.append(oof_lgb)
    all_test.append(test_lgb)
    all_scores.append(sc_lgb)
    print(f"LGB-s{seed}: {np.mean(sc_lgb):.6f}")

    # --- CatBoost ---
    oof_cat  = np.zeros(len(X_train))
    test_cat = np.zeros(len(X_test))
    sc_cat   = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_tr_e, X_val_e, X_te_e = target_encode(
            X_train.iloc[tr_idx], y_train.iloc[tr_idx],
            X_train.iloc[val_idx], X_test, TE_COLS)

        m = CatBoostClassifier(
            iterations=10000, depth=5, learning_rate=0.01, l2_leaf_reg=10,
            min_data_in_leaf=50, random_state=seed, verbose=False,
            thread_count=-1, early_stopping_rounds=200, eval_metric='AUC'
        )
        m.fit(X_tr_e, y_train.iloc[tr_idx],
              eval_set=(X_val_e, y_train.iloc[val_idx]), verbose=False)
        oof_cat[val_idx] = m.predict_proba(X_val_e)[:, 1]
        test_cat += m.predict_proba(X_te_e)[:, 1] / n_folds
        sc_cat.append(roc_auc_score(y_train.iloc[val_idx], oof_cat[val_idx]))

    all_oof.append(oof_cat)
    all_test.append(test_cat)
    all_scores.append(sc_cat)
    print(f"CAT-s{seed}: {np.mean(sc_cat):.6f}")

    # --- XGBoost ---
    oof_xgb  = np.zeros(len(X_train))
    test_xgb = np.zeros(len(X_test))
    sc_xgb   = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_tr_e, X_val_e, X_te_e = target_encode(
            X_train.iloc[tr_idx], y_train.iloc[tr_idx],
            X_train.iloc[val_idx], X_test, TE_COLS)

        m = xgb.XGBClassifier(
            n_estimators=10000, max_depth=4, learning_rate=0.01, subsample=0.7,
            colsample_bytree=0.7, min_child_weight=10, reg_alpha=0.1,
            reg_lambda=5.0, gamma=0.2, random_state=seed, n_jobs=-1,
            eval_metric='auc', early_stopping_rounds=200
        )
        m.fit(X_tr_e, y_train.iloc[tr_idx],
              eval_set=[(X_val_e, y_train.iloc[val_idx])], verbose=False)
        oof_xgb[val_idx] = m.predict_proba(X_val_e)[:, 1]
        test_xgb += m.predict_proba(X_te_e)[:, 1] / n_folds
        sc_xgb.append(roc_auc_score(y_train.iloc[val_idx], oof_xgb[val_idx]))

    all_oof.append(oof_xgb)
    all_test.append(test_xgb)
    all_scores.append(sc_xgb)
    print(f"XGB-s{seed}: {np.mean(sc_xgb):.6f}\n")

print("Training complete!")

## Step 4: Ensemble Methods & Blending with V16

In [ ]:
# Evaluate ensemble methods
def rank_avg(preds_list):
    r = np.zeros(len(preds_list[0]))
    for p in preds_list:
        r += pd.Series(p).rank(pct=True).values
    return r / len(preds_list)

simple_oof  = np.mean(all_oof, axis=0)
simple_test = np.mean(all_test, axis=0)
simple_auc  = roc_auc_score(y_train, simple_oof)

rank_oof  = rank_avg(all_oof)
rank_test = rank_avg(all_test)
rank_auc  = roc_auc_score(y_train, rank_oof)

meta = LogisticRegression(C=0.01, max_iter=1000, random_state=42)
mtr  = pd.DataFrame({f'm{i}': all_oof[i]  for i in range(len(all_oof))})
mte  = pd.DataFrame({f'm{i}': all_test[i] for i in range(len(all_test))})
meta.fit(mtr, y_train)
meta_oof  = meta.predict_proba(mtr)[:, 1]
meta_test = meta.predict_proba(mte)[:, 1]
meta_auc  = roc_auc_score(y_train, meta_oof)

print(f"Simple Average AUC: {simple_auc:.6f}")
print(f"Rank Average AUC:   {rank_auc:.6f}")
print(f"Meta-model AUC:     {meta_auc:.6f}")

# Select best
scores_dict = {
    'simple': (simple_auc, simple_test),
    'rank':   (rank_auc,   rank_test),
    'meta':   (meta_auc,   meta_test),
}
best_name = max(scores_dict, key=lambda k: scores_dict[k][0])
best_auc, new_preds = scores_dict[best_name]
print(f"\n✅ Best ensemble: {best_name.upper()} (AUC: {best_auc:.6f})")

# Blend with V16
print("\n" + "="*50)
print("BLENDING with V16 (A.csv):")
print("="*50)

def rank_norm(preds):
    return pd.Series(preds).rank(pct=True).values

new_preds_rn  = rank_norm(new_preds)
prev_preds_rn = rank_norm(prev_preds)

# Test multiple blend ratios
for w_prev in [0.3, 0.4, 0.5, 0.6]:
    w_new  = 1.0 - w_prev
    blend  = w_prev * prev_preds_rn + w_new * new_preds_rn
    sub = pd.DataFrame({
        'id': test_ids,
        'Heart Disease': np.clip(blend, 0, 1)
    })
    sub.to_csv(f'blend_{w_prev}.csv', index=False)
    print(f"Saved blend_{w_prev}.csv")

# Main submission: 50/50 blend
final_blend = 0.5 * prev_preds_rn + 0.5 * new_preds_rn
submission = pd.DataFrame({
    'id': test_ids,
    'Heart Disease': np.clip(final_blend, 0, 1)
})
submission.to_csv('submission.csv', index=False)

print(f"\n✅ Main submission saved (50/50 blend)")
print(submission.head())